<a href="https://colab.research.google.com/github/Hydrometeorological-Remote-Sensing/cropmapping/blob/sw_ww_test/2_prediction_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data preparation

In [ ]:
from google.colab import auth
auth.authenticate_user()
import ee
ee.Authenticate()
ee.Initialize()
import tensorflow as tf
import folium
import numpy as np
from datetime import datetime, timedelta, date
import random
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(4).int()
    cloudsBitMask = ee.Number(2).pow(3).int()
    qa = image.select('QA_PIXEL')
    mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0))
    mask2 = image.mask().reduce('min')
    # mask3 = image.select(opticalBands).gt(0).And(
    #     image.select(opticalBands).lt(10000)).reduce('min')
    mask = mask1.And(mask2)#.And(mask3)
    return image.updateMask(mask)
# Cloud masking function for s2
def maskClouds(img):
  clouds = ee.Image(img.get('cloud_mask')).select('probability')
  isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
  return img.updateMask(isNotCloud)
def maskEdges(s2_img):
  return s2_img.updateMask(
      s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))
def merge_s2_l8(s2,l8):
  merged = ee.ImageCollection([s2,l8]).mean()
  return merged
def s2_vi(s2):
  s2 = s2.addBands(s2.normalizedDifference(['nir', 'red']).rename('ndvi'))
  s2 = s2.addBands(s2.select(['ndvi']).multiply(s2.select(['nir'])).rename('nirv'))
  return s2

In [ ]:
def indexJoin(collectionA, collectionB, propertyName):
  joined = ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(
    primary = collectionA,
    secondary = collectionB,
    condition = ee.Filter.equals(
      leftField = 'system:index',
      rightField = 'system:index'
    )
  ))
  # Merge the bands of the joined image.
  return joined.map(lambda image: image.addBands(ee.Image(image.get(propertyName))))

In [ ]:
us = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter(
    'ADM0_NAME == "United States of America"'
)
states = [
    "Washington",
    "Oregon",
    "Idaho",
    "Montana"
]

trainingPolys = us.filter(ee.Filter.inList("ADM1_NAME", states))
trainingPolysList = trainingPolys.toList(trainingPolys.size())

In [ ]:
    "North Dakota",
    "Montana",
    "Minnesota"

In [ ]:
polyImage = ee.Image(0).byte().paint(trainingPolys, 1)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['yellow', 'blue']})
map = folium.Map(location=[0, -60], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data © Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Define year

In [ ]:
bandNamesOut = ['blue','green','red','nir','swir1','swir2']
bandNamesl8 = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'];
bandNamesS2 = ['B2','B3','B4','B8','B11','B12'];
BANDS = ['blue','green','red','nir','swir1','swir2','ndvi','nirv']
RESPONSE = 'cropland'
FEATURES = BANDS  + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

MAX_CLOUD_PROBABILITY = 65
l8sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(maskL8sr)
l8sr = l8sr.select(bandNamesl8,bandNamesOut)

s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').map(maskEdges)
list1 = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list1, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

In [ ]:
YEAR = 2022
dte1 = date(YEAR, 6, 1)
l8_dte1 = date(YEAR, 7, 31)
s2_dte1 = date(YEAR, 7, 31)
l8_criteria = ee.Filter.date(dte1.strftime('%Y-%m-%d'),l8_dte1.strftime('%Y-%m-%d'))
s2_criteria = ee.Filter.date(dte1.strftime('%Y-%m-%d'),s2_dte1.strftime('%Y-%m-%d'))
l8_reduced = l8sr.filter(l8_criteria).median().multiply(0.0000275).add(-0.2).float()

s2SrWithCloudMask = indexJoin(s2Sr.filter(s2_criteria), s2Clouds.filter(s2_criteria), 'cloud_mask')
s2_reduced = s2SrWithCloudMask.map(maskClouds)
s2_reduced = s2_reduced.select(bandNamesS2,bandNamesOut).median().divide(10000).float()

image = merge_s2_l8(s2_reduced,l8_reduced)
image = s2_vi(image).float()

## Export prediciton

In [ ]:
def doExport(out_image_base, kernel_buffer, region,folder):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toDrive(
    image = image,
    folder = folder,
    description = out_image_base,
    fileNamePrefix = out_image_base,
    region = region.geometry(),#.getInfo()['coordinates'],
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  print('Running image export to Cloud Storage...')
  task.start()

  # Block until the task completes.
  

In [ ]:
"""# Data Preparation"""
# Specify names locations for outputs in Cloud Storage. 

for state in states:
  FOLDER = 'prediction_sw_67_{}'.format(state)
  bj_image_base = 'prediction_sw_67_{}'.format(state)
  # Half this will extend on the sides of each patch.
  bj_kernel_buffer = [128, 128]
  trainingPoly = us.filter(ee.Filter.inList("ADM1_NAME",[state])) 
  doExport(bj_image_base, bj_kernel_buffer,trainingPoly,FOLDER)

